In [2]:
from PIL import Image
import numpy as np
c0 = Image.open('data/test/0/1.png')
display(c0)
c0_numpy = np.array(c0)
c0_numpy.shape

(28, 28, 4)

In [3]:
c0_numpy[:, :, :-1].shape

(28, 28, 3)

# 1. CNN

In [5]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from PIL import Image
np.random.seed(3)

In [7]:
circle = Image.open('data/handwriting/handwriting_shape/train/circle/circle001.png')
np.array(circle).shape
# 모든 이미지의 shape 같을 필요는 없음

(24, 24, 4)

In [12]:
# 1. 데이터 생성하기 학습훈련셋 : (data/handwriting/handwriting_shape/train/*) 
                # 테스트 셋 : (data/handwriting/handwriting_shape/test/*) 
train_datagen = ImageDataGenerator(rescale=1./255) # 0~1사이의 값으로 스케일 조정
train_data = train_datagen.flow_from_directory( 
    'data/handwriting/handwriting_shape/train/',# train 아래 폴더명이 라벨
    target_size=(24,24), # 넘파이 배열의 이미지 크기 조정
    class_mode='categorical', # 원핫인코딩 형식으로 반환
    batch_size=3 # 한번에 반환할 이미지와 라벨수 
)
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    'data/handwriting/handwriting_shape/test/',
    target_size=(24,24),
    class_mode='categorical',
    batch_size=3,
    shuffle=False # 이미지와 라벨 순서가 고정
)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [13]:
test_data

In [14]:
X_images, Y_labels = next(test_data) # 한 배치 사이즈 만큼 데이터 가져오기
X_images

array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]],


       [[[1., 1., 1.],
         [1., 1., 1.],
         [1., 

In [15]:
print(test_data.n) # 데이터 전체 갯수
print(test_data.batch_size) # 배치 사이즈

15
3


In [17]:
# 전체 가져오기
all_images = [] # 독립변수
all_labels = [] # 종속변수
for images, labels in test_data:
    all_images.append(images)
    all_labels.append(labels)
    if len(all_images) * test_data.batch_size >= test_data.n:
        break
X_test_images = np.concatenate(all_images)
Y_test_labels = np.concatenate(all_labels)

print(X_test_images.shape)
print(Y_test_labels.shape)

(15, 24, 24, 3)
(15, 3)


In [24]:
# 2. 모델구성
model = Sequential()
model.add(Conv2D(32,
                kernel_size=(3,3),
                activation='relu',
                input_shape=(24,24,3)))
model.add(Conv2D(64,
                kernel_size=(3,3),
                activation='relu',
                ))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten()) # 3차원 데이터를 1차원으로
model.add(Dense(units=128,
               activation='relu'))
model.add(Dense(units=3,
               activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 22, 22, 32)        896       
                                                                 
 conv2d_5 (Conv2D)           (None, 20, 20, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 6400)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               819328    
                                                                 
 dense_5 (Dense)             (None, 3)                 387       
                                                      

In [25]:
# 3. 학습설정
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [26]:
# 4. 학습시키기
model.fit(train_data,
         epochs=50,
         validation_data=test_data)

Epoch 1/50
15/15 [==============================] - 1s 22ms/step - loss: 1.0440 - accuracy: 0.5333 - val_loss: 0.6292 - val_accuracy: 0.9333
Epoch 2/50
15/15 [==============================] - 0s 8ms/step - loss: 0.2921 - accuracy: 0.9111 - val_loss: 0.1109 - val_accuracy: 1.0000
Epoch 3/50
15/15 [==============================] - 0s 7ms/step - loss: 0.0322 - accuracy: 1.0000 - val_loss: 0.0415 - val_accuracy: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 7ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0099 - val_accuracy: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 7ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0070 - val_accuracy: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 7ms/step - loss: 5.3034e-04 - accuracy: 1.0000 - val_loss: 0.0061 - val_accuracy: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 7ms/step - loss: 2.4855e-04 - accuracy: 1.0000 - val_loss: 0.0078 - val_accuracy: 1.0000
Epoch 8/50


In [27]:
# 모델 평가
loss, accuracy = model.evaluate(test_data)
print(loss)
print(accuracy)

5/5 [==============================] - 0s 6ms/step - loss: 0.0030 - accuracy: 1.0000
0.002985743572935462
1.0


In [29]:
# 예측하기
yhat = model.predict(test_data)
yhat.argmax(axis=1)

5/5 [==============================] - 0s 4ms/step


array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2], dtype=int64)

In [32]:
np.set_printoptions(formatter={'float':lambda x : "{:0.2f}".format(x)})
yhat

array([[1.00, 0.00, 0.00],
       [1.00, 0.00, 0.00],
       [1.00, 0.00, 0.00],
       [1.00, 0.00, 0.00],
       [1.00, 0.00, 0.00],
       [0.00, 1.00, 0.00],
       [0.00, 1.00, 0.00],
       [0.00, 1.00, 0.00],
       [0.00, 1.00, 0.00],
       [0.01, 0.96, 0.03],
       [0.00, 0.00, 1.00],
       [0.00, 0.00, 1.00],
       [0.00, 0.00, 1.00],
       [0.00, 0.00, 1.00],
       [0.00, 0.00, 1.00]], dtype=float32)

In [30]:
# 실제 데이터
test_data.labels

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

# 2. 데이터 증강을 통한 CNNaccuracy 올리기
- 데이터 추가 확보
- 이미지 데이터를 증강(데이터 늘리기) : train set에서만 
- 레이어층(Conv2D, MaxPooling, 배치정규화, Dense)
- 활성화함수, 옵티마이저, Dropout(과적합 다소 해결)

In [33]:
# 1. 데이터 생성
train_datagen = ImageDataGenerator(rescale=1./255, # 스케일 조정
                                  rotation_range=10, # 이미지를 최대 ±10도 회전
                                  width_shift_range=0.2, # 가로 방향으로 최대 20% 이동
                                  height_shift_range=0.2, # 세로 방향으로 최대 20% 이동
                                  fill_mode='nearest', # 이미지 확대시 채우는 방식
                                  shear_range=0.7, # 최대 0.7 라디안(40도)으로 이미지 밀림
                                  zoom_range=[0.9, 1.5], # 90~150% 크기로 확대/축소
                                  horizontal_flip=True, # 수평 방향 뒤집기
                                  vertical_flip=True, # 수직 방향 뒤집기
                                  
                                  )
train_data = train_datagen.flow_from_directory(
        'data/handwriting/handwriting_shape/train',
        target_size=(24,24),
        class_mode='categorical',
        batch_size=3)
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    'data/handwriting/handwriting_shape/test',
        target_size=(24,24),
        class_mode='categorical',
        batch_size=3,
#         shuffle=True  기본값
)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [34]:
# 2. 모델구성
model = Sequential()
model.add(Conv2D(64,
                kernel_size=(3,3),
                activation='relu',
                input_shape=(24,24,3)))
model.add(Conv2D(64,
                kernel_size=(3,3),
                activation='relu',
                ))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,
                kernel_size=(3,3),
                activation='relu',
                input_shape=(24,24,3)))
model.add(Conv2D(64,
                kernel_size=(3,3),
                activation='relu',
                ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten()) # 3차원 데이터를 1차원으로
model.add(Dense(units=128,
               activation='relu'))
model.add(Dense(units=3,
               activation='softmax'))
# model.summary()
# 3. 학습설정
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
# 4. 학습시키기
model.fit(train_data,
         epochs=50,
         validation_data=test_data)

Epoch 1/50
15/15 [==============================] - 1s 21ms/step - loss: 1.1528 - accuracy: 0.2444 - val_loss: 1.0968 - val_accuracy: 0.4000
Epoch 2/50
15/15 [==============================] - 0s 9ms/step - loss: 1.1101 - accuracy: 0.3333 - val_loss: 1.0908 - val_accuracy: 0.3333
Epoch 3/50
15/15 [==============================] - 0s 8ms/step - loss: 1.0914 - accuracy: 0.3778 - val_loss: 1.0945 - val_accuracy: 0.3333
Epoch 4/50
15/15 [==============================] - 0s 8ms/step - loss: 1.1101 - accuracy: 0.2889 - val_loss: 1.0934 - val_accuracy: 0.3333
Epoch 5/50
15/15 [==============================] - 0s 8ms/step - loss: 1.0830 - accuracy: 0.5111 - val_loss: 1.0670 - val_accuracy: 0.4667
Epoch 6/50
15/15 [==============================] - 0s 8ms/step - loss: 1.0320 - accuracy: 0.5778 - val_loss: 1.0904 - val_accuracy: 0.4000
Epoch 7/50
15/15 [==============================] - 0s 10ms/step - loss: 0.9714 - accuracy: 0.4889 - val_loss: 1.2530 - val_accuracy: 0.4000
Epoch 8/50
15/15 [

In [36]:
# 평가하기
score = model.evaluate(test_data)
print('정확도(accuracy) :', score[1])

5/5 [==============================] - 0s 6ms/step - loss: 0.0012 - accuracy: 1.0000
정확도(accuracy) : 1.0


In [37]:
pred = model.predict(test_data)
print(pred)

5/5 [==============================] - 0s 3ms/step
[[0.00 1.00 0.00]
 [1.00 0.00 0.00]
 [0.00 0.00 1.00]
 [1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 1.00 0.00]
 [1.00 0.00 0.00]
 [1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]
 [0.00 0.00 1.00]
 [0.00 0.00 1.00]
 [0.01 0.00 0.98]
 [0.00 1.00 0.00]
 [1.00 0.00 0.00]]


In [39]:
pred.argmax(axis=1)

array([1, 0, 2, 0, 1, 1, 0, 0, 1, 2, 2, 2, 2, 1, 0], dtype=int64)